# Install Java, Spark, and Findspark
This installs Apache Spark 2.2.1, Java 8, and [Findspark](https://github.com/minrk/findspark), a library that makes it easy for Python to find Spark.

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz

In [0]:
!tar xf spark-2.4.0-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
Set the locations where Spark and Java are installed.

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"

# Start a SparkSession
This will start a local Spark session.

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [0]:
from pyspark import SparkContext, SparkConf

In [0]:
conf = SparkConf().setAppName("Scipy").setMaster("local[*]")
sc = SparkContext(conf=conf)

#**Dataset: - PubG : Introduction to Dataframe **

Cloning data from github Repository :-  https://github.com/Heisenberg0203/PUBG.git

In [0]:
!git clone https://github.com/Heisenberg0203/PUBG.git

In [0]:
!ls PUBG/

In [0]:
dataset = sc.textFile('PUBG/PUBG_Player_Statistics.csv')

In [0]:
dataset.collect()[0:5]

### Kill Count

In [0]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [0]:
dataset = sqlContext.read.format("com.databricks.spark.csv").options(header='true',inferschema='true').load('PUBG/PUBG_Player_Statistics.csv')

In [0]:
dataset.printSchema()

####Top 10 Player who have killed the most in Solo Match

In [0]:
solokill=dataset.select(['player_name','solo_Kills'])
solokill.take(5)

In [0]:
from pyspark.sql.functions import desc

In [0]:
solokill.sort(desc('solo_Kills')).show(10)

In [0]:
import matplotlib.pyplot as plt

In [0]:

a=solokill.sort(desc('solo_Kills')).limit(10)

In [0]:
a = a.toPandas()

In [0]:
plt.bar(x=a.player_name,height=a.solo_Kills)
plt.xticks(rotation=90)
plt.show()

In [0]:
#@title Exercise
##Plot Duo_kills of the Player

### Player with maximum solo win ratio

In [0]:
solowin=dataset.select(['player_name','solo_WinRatio'])
solowin.take(5)

In [0]:
solowin.sort(desc('solo_WinRatio')).show(10)

### Similarly we can find for duo_WinRatio and squad_WinRatio

In [0]:
duowin=dataset.select(['player_name','duo_WinRatio'])
duowin.take(5)
duowin.sort(desc('duo_WinRatio')).show(10)

In [0]:
squadwin=dataset.select(['player_name','squad_WinRatio'])
squadwin.take(5)
squadwin.sort(desc('squad_WinRatio')).show(10)

## Player with maximum solo,duo,squad ratio

In [0]:
win=dataset.select(['player_name','solo_WinRatio','duo_WinRatio','squad_WinRatio'])
win.take(5)

In [0]:
winf=win.withColumn('total', sum(win[col] for col in ['solo_WinRatio','duo_WinRatio','squad_WinRatio']))
winf.take(5)

In [0]:
winf.sort(desc('total')).show(10)


In [0]:
wint=winf.sort(desc('total')).limit(10)
wint = wint.toPandas()

In [0]:
wint[['solo_WinRatio','duo_WinRatio','squad_WinRatio']].plot(kind='bar',stacked='True')

In [0]:
#@title Exercise
##Which ride(Walk or Ride) is prefereed by Top 10 Players

###Kills and Heals vs Win Ratio

In [0]:
multigraph = dataset.select(['solo_Kills','solo_Heals','solo_Revives','solo_Boosts'])
killdata = multigraph.groupby(['solo_Kills']).count()
healdata = multigraph.groupby(['solo_Heals']).count()
##sort in descending order
sortedkilldata = killdata.sort(desc('count'))
sortedhealdata = healdata.sort(desc('count'))

In [0]:
killdf=sortedkilldata.toPandas()
healdf=sortedhealdata.toPandas()

In [0]:
print(killdf.head(5))
print(healdf.head(5))

In [0]:
plt.subplot(1,2,1)
plt.plot(killdf['solo_Kills'],killdf['count'])
plt.xlabel('No_of_Kills')
plt.ylabel('WIN_RATIO')
plt.subplot(1,2,2)
plt.xlabel('No_of_Heals')
plt.ylabel('WIN_RATIO')
plt.plot(healdf['solo_Heals'],healdf['count'])

In [0]:
#@title Exercise
##Show No_of Revivies and No_Of Boosts in Overall Category

### Lets analyze the player with max solo_WinRatio using SparkSQL

In [0]:
dataset.registerTempTable("dataset_table")
df=sqlContext.sql('SELECT MAX(solo_WinRatio) FROM dataset_table')
df.show()

In [0]:
sqlContext.sql('SELECT * FROM dataset_table WHERE solo_WinRatio = 100 ' ).show()

In [0]:
from pyspark.sql import functions as func
##winRatio vs solo_kills
df1 = dataset.select(['solo_KillsPg','solo_WinRatio']).withColumn('solo_KillsPg',func.round(dataset['solo_KillsPg'],0))
df1 = df1.groupby(['solo_KillsPg']).agg(func.avg('solo_WinRatio').alias('WinPerc'))
##winRatio vs duo_kills
df2 = dataset.select(['duo_KillsPg','duo_WinRatio']).withColumn('duo_KillsPg',func.round(dataset['duo_KillsPg'],0))
df2 = df2.groupby(['duo_KillsPg']).agg(func.avg('duo_WinRatio').alias('WinPerc'))
##winRatio vs squad_Kills
df3 = dataset.select(['squad_KillsPg','squad_WinRatio']).withColumn('squad_KillsPg',func.round(dataset['squad_KillsPg'],0))
df3 = df3.groupby(['squad_KillsPg']).agg(func.avg('squad_WinRatio').alias('WinPerc'))

In [0]:
df2.collect()

In [0]:
pdf1 = df1.toPandas().sort_values(by='solo_KillsPg')
pdf2 = df2.toPandas().sort_values(by='duo_KillsPg')
pdf3 = df3.toPandas().sort_values(by='squad_KillsPg')

plt.plot(pdf1['solo_KillsPg'],pdf1['WinPerc'],c='b')
plt.plot(pdf2['duo_KillsPg'],pdf2['WinPerc'],c='g')
plt.plot(pdf3['squad_KillsPg'],pdf3['WinPerc'],c='r')
plt.text(15,30,'Solos',color='b',fontsize = 17,style = 'italic')
plt.text(15,20,'Duos',color='g',fontsize = 17,style = 'italic')
plt.text(15,10,'Squads',color='r',fontsize = 17,style = 'italic')
plt.xlabel('Number of kills',)
plt.ylabel('Win Percentage',)
plt.title('Solo vs Duo vs Squad Kills',)
plt.show()

##Pandas vs Spark

In [0]:
##Pandas vs Spark
from pyspark.sql.functions import col
import time
tic = time.time()
sparkdata = sqlContext.read.format("com.databricks.spark.csv").options(header='true',inferschema='true').load('PUBG/PUBG_Player_Statistics.csv').repartition(8)
subset = sparkdata.select(['player_name','squad_Kills'])
filterdata = subset.filter((col('squad_Kills')<5))
sorteddata = filterdata.sort('squad_Kills')
output = sorteddata.limit(10)
toc =time.time()
print(toc-tic)

In [0]:
import pandas as pd
tic = time.time()
dataframe = pd.read_csv('PUBG/PUBG_Player_Statistics.csv')
subset = dataframe[['player_name','squad_Kills']]
filterdata = subset[subset['squad_Kills']<5]
sorteddata = filterdata.sort_values(by='squad_Kills')
output = sorteddata.head(10)
toc = time.time()
print(toc-tic)
                